In [1]:
#essential libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import glob
import time
import re
import seaborn as sns

### Gathering data

In [2]:
# downloading the WeRateDogs Twitter archive data (twitter_archive_enhanced.csv and reading them.
twitter_archive = pd.read_csv("twitter-archive-enhanced.csv")
#twitter_archive.info()

In [3]:
# download the tweet image prediction (image_predictions.tsv) an reading
import requests
import io
#dowloading the data
r = requests.get('https://d17h27t6h515a5.cloudfront.net/topher/2017/August/599fd2ad_image-predictions/image-predictions.tsv')
# reading data as dataframe
twitt_img_pred = pd.read_csv(io.StringIO(r.text),sep='\t')

In [ ]:
#twitt_img_pred.head()

In [ ]:
# connection to Twitter API
import tweepy

consumer_key = '****'
consumer_secret = '******'
access_token = '******'
access_secret = '*******'

auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token,access_secret)

api = tweepy.API(auth,wait_on_rate_limit = True)

In [ ]:
valid_list = []
invalid_list= []

start = time.time()

# adding each favorites and retwwets count in a dataframe
for tweet_id in twitter_archive['tweet_id']:
    try:
        tweet = api.get_status(tweet_id, tweet_mode='extended')._json 
        favorites = tweet['favorite_count'] # tweet's favorites
        retweets = tweet['retweet_count'] # retweets count
        valid_list.append({'tweet_id': int(tweet_id), 'favorites': int(favorites), 'retweets': int(retweets)}) # appending by desired value
        
    except Exception as e:
        print(str(tweet_id)+ " __ " + str(e))
        invalid_list.append(tweet_id) # append list when there is no status found on the id
        

end = time.time()
print(end - start)



In [ ]:
#writing it on csv text file
tweets_json = pd.DataFrame(valid_list, columns= ['tweet_id', 'favorites', 'retweets'], index_col = False )
tweets_json.to_csv('tweets_json.txt')

In [17]:
tweets_data = pd.read_csv('tweets_json.txt', encoding = 'utf-8')

### ASSESING DATA

In [4]:
#Visual assesement
twitter_archive.sample(15)

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
1243,711968124745228288,NaN,NaN,2016-03-21 17:30:03 +0000,"<a href=""http://twitter.com/download/iphone"" r...",Meet Winston. He's trapped in a cup of coffee....,NaN,NaN,NaN,https://twitter.com/dog_rates/status/711968124...,10,10,Winston,None,None,pupper,None
1474,693644216740769793,6.936422e+17,4.196984e+09,2016-01-31 03:57:23 +0000,"<a href=""http://twitter.com/download/iphone"" r...",BREAKING PUPDATE: I've just been notified that...,NaN,NaN,NaN,NaN,10,10,None,None,None,None,None
663,790946055508652032,NaN,NaN,2016-10-25 16:00:09 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Betty. She's assisting with the dishes...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/790946055...,12,10,Betty,None,None,None,puppo
1504,691820333922455552,NaN,NaN,2016-01-26 03:09:55 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Brockly. He's an uber driver. Falls as...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/691820333...,8,10,Brockly,None,None,None,None
1087,738156290900254721,NaN,NaN,2016-06-01 23:52:28 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Storkson. He's wet and sad. 10/10 chee...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/738156290...,10,10,Storkson,None,None,None,None
1651,683498322573824003,NaN,NaN,2016-01-03 04:01:13 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Griffin. He's desperate for both a phy...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/683498322...,11,10,Griffin,None,None,None,None
1001,747933425676525569,NaN,NaN,2016-06-28 23:23:19 +0000,"<a href=""http://twitter.com/download/iphone"" r...",Pls don't send more sherks. I don't care how s...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/747933425...,11,10,None,None,None,None,None
748,779377524342161408,NaN,NaN,2016-09-23 17:50:56 +0000,"<a href=""http://twitter.com/download/iphone"" r...",Have you ever seen such a smol pupper? Portabl...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/779377524...,12,10,None,None,None,pupper,None
1239,712092745624633345,NaN,NaN,2016-03-22 01:45:15 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Steven. He's inverted af. Also very he...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/712092745...,7,10,Steven,None,None,None,None
824,769940425801170949,NaN,NaN,2016-08-28 16:51:16 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Klein. These pics were taken a month a...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/769940425...,12,10,Klein,None,None,None,None


In [5]:
twitter_archive.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2356 entries, 0 to 2355
Data columns (total 17 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   tweet_id                    2356 non-null   int64  
 1   in_reply_to_status_id       78 non-null     float64
 2   in_reply_to_user_id         78 non-null     float64
 3   timestamp                   2356 non-null   object 
 4   source                      2356 non-null   object 
 5   text                        2356 non-null   object 
 6   retweeted_status_id         181 non-null    float64
 7   retweeted_status_user_id    181 non-null    float64
 8   retweeted_status_timestamp  181 non-null    object 
 9   expanded_urls               2297 non-null   object 
 10  rating_numerator            2356 non-null   int64  
 11  rating_denominator          2356 non-null   int64  
 12  name                        2356 non-null   object 
 13  doggo                       2356 

In [6]:
twitter_archive.describe()

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,retweeted_status_id,retweeted_status_user_id,rating_numerator,rating_denominator
count,2.356000e+03,7.800000e+01,7.800000e+01,1.810000e+02,1.810000e+02,2356.000000,2356.000000
mean,7.427716e+17,7.455079e+17,2.014171e+16,7.720400e+17,1.241698e+16,13.126486,10.455433
std,6.856705e+16,7.582492e+16,1.252797e+17,6.236928e+16,9.599254e+16,45.876648,6.745237
min,6.660209e+17,6.658147e+17,1.185634e+07,6.661041e+17,7.832140e+05,0.000000,0.000000
25%,6.783989e+17,6.757419e+17,3.086374e+08,7.186315e+17,4.196984e+09,10.000000,10.000000
50%,7.196279e+17,7.038708e+17,4.196984e+09,7.804657e+17,4.196984e+09,11.000000,10.000000
75%,7.993373e+17,8.257804e+17,4.196984e+09,8.203146e+17,4.196984e+09,12.000000,10.000000
max,8.924206e+17,8.862664e+17,8.405479e+17,8.874740e+17,7.874618e+17,1776.000000,170.000000


In [7]:
twitt_img_pred.sample(10)

,tweet_id,jpg_url,img_num,p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog
403,673700254269775872,https://pbs.twimg.com/media/CVl2ydUWsAA1jD6.jpg,1,water_bottle,0.614536,False,ashcan,0.050911,False,bucket,0.037432,False
1159,733828123016450049,https://pbs.twimg.com/media/Ci8UxxcW0AYgHDh.jpg,2,beagle,0.472324,True,Walker_hound,0.121779,True,Saint_Bernard,0.114640,True
743,687476254459715584,https://pbs.twimg.com/media/CYpoAZTWEAA6vDs.jpg,1,wood_rabbit,0.702725,False,Angora,0.190659,False,hare,0.105072,False
1470,779123168116150273,https://pbs.twimg.com/media/CtAAYizW8AAWzBZ.jpg,1,toy_poodle,0.431080,True,soft-coated_wheaten_terrier,0.060365,True,cocker_spaniel,0.059845,True
1724,819952236453363712,https://pbs.twimg.com/media/C2EONHNWQAUWxkP.jpg,1,American_Staffordshire_terrier,0.925505,True,Staffordshire_bullterrier,0.036221,True,Italian_greyhound,0.020412,True
279,671109016219725825,https://pbs.twimg.com/media/CVBCFkyU4AE2Wcr.jpg,1,basenji,0.855959,True,beagle,0.036723,True,toy_terrier,0.029258,True
1447,776088319444877312,https://pbs.twimg.com/media/CsU4NKkW8AUI5eG.jpg,3,web_site,0.999916,False,pug,0.000077,True,menu,0.000002,False
1180,738166403467907072,https://pbs.twimg.com/media/Cj5-aUQUgAAb43p.jpg,2,keeshond,0.878886,True,Norwegian_elkhound,0.086659,True,malamute,0.021280,True
1669,813066809284972545,https://pbs.twimg.com/media/C0iX8OOVEAEIpMC.jpg,1,toy_terrier,0.776400,True,Pembroke,0.115034,True,basenji,0.048873,True
1309,754011816964026368,https://pbs.twimg.com/media/CnbJuPoXEAAjcVF.jpg,1,French_bulldog,0.600985,True,Boston_bull,0.273176,True,boxer,0.056772,True


In [8]:
twitt_img_pred.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2075 entries, 0 to 2074
Data columns (total 12 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   tweet_id  2075 non-null   int64  
 1   jpg_url   2075 non-null   object 
 2   img_num   2075 non-null   int64  
 3   p1        2075 non-null   object 
 4   p1_conf   2075 non-null   float64
 5   p1_dog    2075 non-null   bool   
 6   p2        2075 non-null   object 
 7   p2_conf   2075 non-null   float64
 8   p2_dog    2075 non-null   bool   
 9   p3        2075 non-null   object 
 10  p3_conf   2075 non-null   float64
 11  p3_dog    2075 non-null   bool   
dtypes: bool(3), float64(3), int64(2), object(4)
memory usage: 152.1+ KB


### quality issues
 
1. (twitter_archive_df) in_reply_to_status_id, in_reply_to_user_id datatype falls either integer or string.
2. (twitter_archive_df) source column is html tagged which is not readble.
3. (twitter_archive_df) timestamp, retweeted_status_timestamp should be converted to datetime format insted of object.
4. (twutter_archive_df) rating numerator and rating denominator has invalid values.
5. (twitter_archive_df) name entry in name column could be invalid (e.g a, an, mo is not vaid names if it is not tranlated from chinise)
6. (twitter_archive_df) in_reply_to_status_id, in_reply_to_user_id, retweeted_status_id, retweeted_status_user_id, retweeted_status_timestamp, expanded_urls has some missing values.
7. (twitter_img_pred) There are (2356-2075=281) missing values in image datsets.
8. (twitter_img_pred) Duplicate tweets has been found on several tweet ids which is actually retweets.


In [9]:
twitter_archive.source.value_counts().sum()

2356

In [10]:
#missing data
twitter_archive.isna().any()

tweet_id                      False
in_reply_to_status_id          True
in_reply_to_user_id            True
timestamp                     False
source                        False
text                          False
retweeted_status_id            True
retweeted_status_user_id       True
retweeted_status_timestamp     True
expanded_urls                  True
rating_numerator              False
rating_denominator            False
name                          False
doggo                         False
floofer                       False
pupper                        False
puppo                         False
dtype: bool

In [11]:
#inavlid nominator and denominator values
twitter_archive['rating_numerator'][twitter_archive['rating_numerator'] >25]

188      420
189      666
290      182
313      960
340       75
433       84
695       75
763       27
902      165
979     1776
1120     204
1202      50
1228      99
1254      80
1274      45
1351      60
1433      44
1634     143
1635     121
1712      26
1779     144
1843      88
2074     420
Name: rating_numerator, dtype: int64

In [12]:
#inavlid nominator and denominator values
twitter_archive['rating_denominator'][twitter_archive['rating_denominator'] >25]

433      70
902     150
1120    170
1202     50
1228     90
1254     80
1274     50
1351     50
1433     40
1634    130
1635    110
1779    120
1843     80
Name: rating_denominator, dtype: int64

In [13]:
#duplicate tweets
twitt_img_pred['jpg_url'].value_counts()

https://pbs.twimg.com/media/CvaYgDOWgAEfjls.jpg    2
https://pbs.twimg.com/media/CsGnz64WYAEIDHJ.jpg    2
https://pbs.twimg.com/media/CsrjryzWgAAZY00.jpg    2
https://pbs.twimg.com/media/Co-hmcYXYAASkiG.jpg    2
https://pbs.twimg.com/media/CUN4Or5UAAAa5K4.jpg    2
                                                  ..
https://pbs.twimg.com/media/Cb78-nOWIAENNRc.jpg    1
https://pbs.twimg.com/media/CXh5_dDWQAIbU-J.jpg    1
https://pbs.twimg.com/media/C2d_vnHWEAE9phX.jpg    1
https://pbs.twimg.com/media/CUntin8WIAADmLk.jpg    1
https://pbs.twimg.com/media/CzaY5UdUoAAC91S.jpg    1
Name: jpg_url, Length: 2009, dtype: int64

In [14]:
# invalid names
twitter_archive['name'][twitter_archive['name'].apply(len) <=2]

56       a
393     Mo
446     Bo
449     Bo
553     Bo
        ..
2349    an
2350     a
2352     a
2353     a
2354     a
Name: name, Length: 79, dtype: object

# Tidiness
1. dog breeds could be stacked into one coulmn rather than in multiple columns
2. unnessary columns which could mege into one column for predictions

# Cleaning

In [18]:

#copying all theree datasets
twitter_archive_clean = twitter_archive.copy()
twitt_img_pred_clean = twitt_img_pred.copy()
tweets_data_clean = tweets_data.copy()


### Define: merging all three dataframe into one dataframe

### Code

In [19]:
df_twitter = (pd.merge(left = twitter_archive_clean, right = tweets_data_clean, 
                      left_on = 'tweet_id', right_on = 'tweet_id',
                     how = 'inner')).merge(twitt_img_pred_clean, on = 'tweet_id', how = 'inner')

### Test

In [20]:
df_twitter.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2057 entries, 0 to 2056
Data columns (total 31 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   tweet_id                    2057 non-null   int64  
 1   in_reply_to_status_id       23 non-null     float64
 2   in_reply_to_user_id         23 non-null     float64
 3   timestamp                   2057 non-null   object 
 4   source                      2057 non-null   object 
 5   text                        2057 non-null   object 
 6   retweeted_status_id         71 non-null     float64
 7   retweeted_status_user_id    71 non-null     float64
 8   retweeted_status_timestamp  71 non-null     object 
 9   expanded_urls               2057 non-null   object 
 10  rating_numerator            2057 non-null   int64  
 11  rating_denominator          2057 non-null   int64  
 12  name                        2057 non-null   object 
 13  doggo                       2057 

### Define: Stacking dog stages into one column

### Code

In [21]:
# concating all dogs which has multiple stages
df_twitter.groupby(['doggo', 'floofer', 'pupper', 'puppo'])[['doggo', 'floofer', 'pupper', 'puppo']].size().reset_index()

,doggo,floofer,pupper,puppo,0
0,None,None,None,None,1739
1,None,None,None,puppo,23
2,None,None,pupper,None,210
3,None,floofer,None,None,7
4,doggo,None,None,None,65
5,doggo,None,None,puppo,1
6,doggo,None,pupper,None,11
7,doggo,floofer,None,None,1


In [22]:
# stacking into all dog stage column into one column
dog_cols = ['doggo', 'floofer', 'pupper', 'puppo']
df_twitter = df_twitter.replace('None','')
df_twitter['stage'] = df_twitter[dog_cols].apply(lambda row: ''.join(row.values.astype(str)), axis=1)
df_twitter['stage']=df_twitter['stage'].replace('', 'None')

In [23]:
#Test 
df_twitter['stage'].value_counts()

None            1739
pupper           210
doggo             65
puppo             23
doggopupper       11
floofer            7
doggopuppo         1
doggofloofer       1
Name: stage, dtype: int64

# Define: Delete unnecessary coulumns and rows 

In [24]:
#df_twitter.columns

### Code

In [25]:
# removing unnecessary rows
df_twitter = df_twitter[pd.isnull(df_twitter['retweeted_status_id'])].reset_index(drop=True) # when you re-run this cell please make sure you have reset your df_twitter

#removing unnecessary columns
drop_columns = ['retweeted_status_id', 'retweeted_status_user_id', 'retweeted_status_timestamp', 'Unnamed: 0', 'img_num','expanded_urls']
df_twitter = df_twitter.drop(drop_columns, axis = 1)




### Test 

In [26]:
#f_twitter.head()
df_twitter.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1986 entries, 0 to 1985
Data columns (total 26 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   tweet_id               1986 non-null   int64  
 1   in_reply_to_status_id  23 non-null     float64
 2   in_reply_to_user_id    23 non-null     float64
 3   timestamp              1986 non-null   object 
 4   source                 1986 non-null   object 
 5   text                   1986 non-null   object 
 6   rating_numerator       1986 non-null   int64  
 7   rating_denominator     1986 non-null   int64  
 8   name                   1986 non-null   object 
 9   doggo                  1986 non-null   object 
 10  floofer                1986 non-null   object 
 11  pupper                 1986 non-null   object 
 12  puppo                  1986 non-null   object 
 13  favorites              1986 non-null   int64  
 14  retweets               1986 non-null   int64  
 15  jpg_

In [27]:
#test
df_twitter.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1986 entries, 0 to 1985
Data columns (total 26 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   tweet_id               1986 non-null   int64  
 1   in_reply_to_status_id  23 non-null     float64
 2   in_reply_to_user_id    23 non-null     float64
 3   timestamp              1986 non-null   object 
 4   source                 1986 non-null   object 
 5   text                   1986 non-null   object 
 6   rating_numerator       1986 non-null   int64  
 7   rating_denominator     1986 non-null   int64  
 8   name                   1986 non-null   object 
 9   doggo                  1986 non-null   object 
 10  floofer                1986 non-null   object 
 11  pupper                 1986 non-null   object 
 12  puppo                  1986 non-null   object 
 13  favorites              1986 non-null   int64  
 14  retweets               1986 non-null   int64  
 15  jpg_

### Define: formating columns into necessary format

### Code

In [28]:
# removing time zone from timestamp
df_twitter['timestamp']= df_twitter['timestamp'].str.slice(start = 0, stop = -6)
# change into date time object
df_twitter['timestamp'] = pd.to_datetime(df_twitter['timestamp'], format = "%Y-%m-%d %H:%M:%S")

### Test 

In [29]:
df_twitter.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1986 entries, 0 to 1985
Data columns (total 26 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   tweet_id               1986 non-null   int64         
 1   in_reply_to_status_id  23 non-null     float64       
 2   in_reply_to_user_id    23 non-null     float64       
 3   timestamp              1986 non-null   datetime64[ns]
 4   source                 1986 non-null   object        
 5   text                   1986 non-null   object        
 6   rating_numerator       1986 non-null   int64         
 7   rating_denominator     1986 non-null   int64         
 8   name                   1986 non-null   object        
 9   doggo                  1986 non-null   object        
 10  floofer                1986 non-null   object        
 11  pupper                 1986 non-null   object        
 12  puppo                  1986 non-null   object        
 13  fav

### Define: dog rating standardization

### Code

In [30]:
#creating a new column named 'rating' for a new standard rating


df_twitter['rating_numerator'] = df_twitter['rating_numerator'].astype(float)

df_twitter['rating_denominator'] = df_twitter['rating_denominator'].astype(float)

df_twitter['dog_rating'] = df_twitter['rating_numerator']/ df_twitter['rating_denominator']

In [31]:
df_twitter['rating_denominator'].value_counts()

10.0     1968
50.0        3
80.0        2
11.0        2
150.0       1
20.0        1
70.0        1
90.0        1
170.0       1
130.0       1
40.0        1
110.0       1
2.0         1
7.0         1
120.0       1
Name: rating_denominator, dtype: int64

### Test

In [32]:
df_twitter.dog_rating

0       1.3
1       1.3
2       1.2
3       1.3
4       1.2
       ... 
1981    0.5
1982    0.6
1983    0.9
1984    0.7
1985    0.8
Name: dog_rating, Length: 1986, dtype: float64

### Define: standardize image prediction values

### Code

In [33]:
# apapend the predition where confidence level has highest value
predictions = []
confidence_level = []

def prediction(df):
    if (df['p1_dog'] == True) & (df['p1_conf'] > df['p2_conf'] ) & (df['p1_conf'] > df['p3_conf'] ):
        predictions.append(df['p1'])
        confidence_level.append(df['p1_conf'])
    elif (df['p2_dog'] == True) & (df['p2_conf'] > df['p1_conf'] ) & (df['p2_conf'] > df['p3_conf'] ):
        predictions.append(df['p2'])
        confidence_level.append(df['p2_conf'])
    elif (df['p3_dog'] == True) & (df['p3_conf'] > df['p1_conf'] ) & (df['p3_conf'] > df['p1_conf'] ):
        predictions.append(df['p3'])
        confidence_level.append(df['p3_conf'])
    else:
        predictions.append('NaN')
        confidence_level.append(0)

df_twitter.apply(prediction, axis=1)
df_twitter['prediction'] = predictions
df_twitter['confidence_level'] = confidence_level

# deleting unnecessinry columns realted to prediction
df_twitter= df_twitter.drop(['p1', 'p1_conf', 'p1_dog', 'p2', 'p2_conf','p2_dog', 'p3', 'p3_conf', 'p3_dog'], axis=1)

### Test

In [34]:
df_twitter.head()

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo,favorites,retweets,jpg_url,stage,dog_rating,prediction,confidence_level
0,892420643555336193,NaN,NaN,2017-08-01 16:23:56,"<a href=""http://twitter.com/download/iphone"" r...",This is Phineas. He's a mystical boy. Only eve...,13.0,10.0,Phineas,,,,,33858,7023,https://pbs.twimg.com/media/DGKD1-bXoAAIAUK.jpg,None,1.3,NaN,0.000000
1,892177421306343426,NaN,NaN,2017-08-01 00:17:27,"<a href=""http://twitter.com/download/iphone"" r...",This is Tilly. She's just checking pup on you....,13.0,10.0,Tilly,,,,,29360,5304,https://pbs.twimg.com/media/DGGmoV4XsAAUL6n.jpg,None,1.3,Chihuahua,0.323581
2,891815181378084864,NaN,NaN,2017-07-31 00:18:03,"<a href=""http://twitter.com/download/iphone"" r...",This is Archie. He is a rare Norwegian Pouncin...,12.0,10.0,Archie,,,,,22081,3487,https://pbs.twimg.com/media/DGBdLU1WsAANxJ9.jpg,None,1.2,Chihuahua,0.716012
3,891689557279858688,NaN,NaN,2017-07-30 15:58:51,"<a href=""http://twitter.com/download/iphone"" r...",This is Darla. She commenced a snooze mid meal...,13.0,10.0,Darla,,,,,36981,7236,https://pbs.twimg.com/media/DF_q7IAWsAEuuN8.jpg,None,1.3,NaN,0.000000
4,891327558926688256,NaN,NaN,2017-07-29 16:00:24,"<a href=""http://twitter.com/download/iphone"" r...",This is Franklin. He would like you to stop ca...,12.0,10.0,Franklin,,,,,35357,7773,https://pbs.twimg.com/media/DF6hr6BUMAAzZgT.jpg,None,1.2,basset,0.555712


### Define:  making source column  more readbale

### code

In [35]:
df_twitter['source'] = df_twitter['source'].apply(lambda x: re.findall(r'>(.*)<', x)[0])

### Test

In [ ]:
df_twitter.info()

### Define: changing datatypyes for each column

### Code

In [ ]:
df_twitter['tweet_id'] = df_twitter['tweet_id'].astype(str)
df_twitter['source'] = df_twitter['source'].astype('category')
df_twitter['favorites'] = df_twitter['favorites'].astype(int)
df_twitter['retweets'] = df_twitter['retweets'].astype(int)
df_twitter['stage'] = df_twitter['stage'].astype('category')
df_twitter['rating_numerator'] = df_twitter['rating_numerator'].astype(float)
df_twitter['rating_denominator'] = df_twitter['tweet_id'].astype(float)
df_twitter['dog_rating'] = df_twitter['dog_rating'].astype(float)


### Test

In [ ]:
df_twitter.info()

# Storing 

In [ ]:
df_twitter.to_csv('twitter_archive_master.csv', encoding = 'utf-8', index =False)

### Analyzing and Visualizing Data

In [ ]:
# reading data as datframe from stored file
df = pd.read_csv('twitter_archive_master.csv')
df.info()

In [ ]:
df.head()

In [ ]:
# changing datatypes for some columns
df['tweet_id'] = df['tweet_id'].astype(str)
df['source'] = df['source'].astype('category')
df['favorites'] = df['favorites'].astype(int)
df['retweets'] = df['retweets'].astype(int)
df['stage'] = df['stage'].astype('category')
df['rating_numerator'] = df['rating_numerator'].astype(float)
df['rating_denominator'] = df['tweet_id'].astype(float)
df['dog_rating'] = df['dog_rating'].astype(float)

In [ ]:
#checking datatypes
df.dtypes

## Most common dog stage

In [ ]:
# extracting data
data = df.groupby('stage')['stage'].count()[1:]

# plotiing intferactive pie chat for exploring most common dog stage
import plotly.express as px
fig = px.pie(df, values=data,names = data.index, color_discrete_sequence=px.colors.diverging.Spectral, hole = .4)
fig.show()

### it seems pupper is the most common dog

### Dog rating in terms of stage

In [ ]:
dog_rating_data = df[df['stage'] != 'None'][['stage','dog_rating']].value_counts().reset_index()
dog_rating_data.rename(columns = {0:'count'}, inplace = True)
fig = px.bar(dog_rating_data, x= 'stage', y= 'count', color = 'dog_rating')
fig.show()

In [ ]:
dog_rating_data.columns

In [ ]:
### Dog breed prediction

In [ ]:

dog_prediction_data = df[df['stage'] != 'None'][['stage','prediction']].value_counts().reset_index()
dog_prediction_data.rename(columns = {0:'count'}, inplace = True)
dog_prediction_data = dog_prediction_data[dog_prediction_data['count']>=2]
fig = px.bar(dog_prediction_data, x= 'stage', y = 'count', color = 'prediction')
fig.show()

In [ ]:
fig_sc = px.scatter(df, 'retweets', 'favorites', color= 'stage', trendline = 'ols')
fig_sc.show()

In [ ]:
fig_line = px.line(df, x='timestamp', y='favorites')
fig_line.show()
# Output html that you can copy paste
fig_line.to_html(full_html=False, include_plotlyjs='cdn')
# Saves a html doc that you can copy paste
fig_line.write_html("output.html", full_html=False, include_plotlyjs='cdn')